# The Hull White Model


In the Hull White model, the short-rate follows the following process.
$$
dr_t = [\theta_t - a r_t]dt + \sigma dW_t
$$
We can instead define the process as,
$$
dx_t = - a x_t dt + \sigma dW_t
$$
where $r_t = \alpha_t + x_t$, i.e. the short rate is the sum of a deterministic $\alpha$ and a stochastic $x$.

In this case, to fit the term structure of interest rates, $\alpha_t$ is given by
$$
\alpha_t = f^M_t + \frac{\sigma^2}{2a^2}(1-e^{-at})^2
$$

[Brigo, Mercurio, Interest Rate Models, 2006 ed., page 73-74].

# PDE for the contract price

Let $V$ be the price of the contract. $V_x$ and $V_{xx}$ are the derivatives w.r.t. $x$.
$$
-V_xax_tdt + V_{xx}\frac{\sigma^2}{2}dt + dV - r V dt = 0
$$


In [ ]:
from sympy import *
from sympy import init_printing
from utils.symfns import discretize_crank_nicolson, find_coefficients

init_printing()

dV, V, Vx, Vxx, sigma, a, r, dx, dt, x = symbols(
    "dV V V_x V_{xx} sigma a r dx, dt, x"
)

We will define the lhs of the equation using sympy as follows:

In [ ]:
model_expr = dV + sigma**2 / 2 * Vxx * dt - a * x * Vx * dt - r * V * dt
model_expr

                                 2     
                      V_{xx}⋅dt⋅σ      
-V⋅dt⋅r - Vₓ⋅a⋅dt⋅x + ──────────── + dV
                           2           

# Crank-Nicolson method.

![cn method image](./assets/cn.png)

In a $3X2$ region of the grid, let $V^u_l$, $V^m_l$, $V^d_l$ be the upper, middle and lower gridpoints on the left hand side, and $V^u_r$, $V^m_r$, $V^d_r$ be the corresponding gridpoints on the right hand side. 

$$
\begin{matrix}
V^u_l & V^u_r \\
V^m_l & V^m_r \\
V^d_l & V^d_r
\end{matrix}
$$

We can substitute $V$, $dV$, $V_x$, and $V_{xx}$ in the above PDE, by the discrete estimates based on the Crank-Nicolson method.

In [ ]:
expr, left_gridpoints, right_gridpoints = discretize_crank_nicolson(
    model_expr, V, dV, Vx, Vxx, dx
)
expr

                          ⎛  V_l__d   V_r__d   V_l__u   V_r__u⎞               
                   a⋅dt⋅x⋅⎜- ────── - ────── + ────── + ──────⎟               
                          ⎝    2        2        2        2   ⎠        ⎛V_l__m
-V_l__m + V_r__m - ──────────────────────────────────────────── - dt⋅r⋅⎜──────
                                       2⋅dx                            ⎝  2   
                                                                              

                                                                            
                 2                                                          
   V_r__m⎞   dt⋅σ ⋅(V_l__d + V_r__d - 2⋅V_l__m - 2⋅V_r__m + V_l__u + V_r__u)
 + ──────⎟ + ───────────────────────────────────────────────────────────────
     2   ⎠                                    2                             
                                          4⋅dx                              

We now need to find the coefficients of $V^u_p$, $V^m_p$, $V^d_p$, $V^u$, $V^m$, $V^d$


To find the coefficient of $V^u_p$, substitute $V^u_p$ to 1, and set the others to zero. Repeat for each of the other values.

In [ ]:
coefs = find_coefficients(expr, left_gridpoints + right_gridpoints)

The PDE is now in the form
$$
Coefs \cdot
\begin{bmatrix}
V^u_l & V^u_r \\
V^m_l & V^m_r \\
V^d_l & V^d_r
\end{bmatrix} = 0
$$

Therefore, if we re-arrange into the form:

$$
A_l \cdot \begin{bmatrix} V^u_l \\ V^m_l \\ V^d_l \end{bmatrix} =
A_r \cdot \begin{bmatrix} V^u_r \\ V^m_r \\ V^d_r \end{bmatrix}
$$

We can run backward induction, by solving for $A_l$ when $A_r$ is known.

$A_l$ is given by the left coefficients, and $A_r$ is given by the negative of the right coefficients.

In [ ]:
for k, v in coefs.items():
    flip = -1 if k in right_gridpoints else 1
    print(f"{k}: {v * flip}")

V^u_l: -a*dt*x/(4*dx) + dt*sigma**2/(4*dx**2)
V^m_l: -dt*r/2 - dt*sigma**2/(2*dx**2) - 1
V^d_l: a*dt*x/(4*dx) + dt*sigma**2/(4*dx**2)
V^u_r: a*dt*x/(4*dx) - dt*sigma**2/(4*dx**2)
V^m_r: dt*r/2 + dt*sigma**2/(2*dx**2) - 1
V^d_r: -a*dt*x/(4*dx) - dt*sigma**2/(4*dx**2)


# Grid Boundary

In the **upper boundary** there is no $V^u_l$ or $V^u_r$.

In [ ]:
expr_up, left_gridpoints, right_gridpoints = discretize_crank_nicolson(
    model_expr, V, dV, Vx, Vxx, dx, "up"
)
coefs_up = find_coefficients(expr_up, left_gridpoints + right_gridpoints)

for k, v in coefs_up.items():
    flip = -1 if k in right_gridpoints else 1
    print(f"{k}: {v * flip}")

V^u_l: 0
V^m_l: -a*dt*x/(2*dx) - dt*r/2 - 1
V^d_l: a*dt*x/(2*dx)
V^u_r: 0
V^m_r: a*dt*x/(2*dx) + dt*r/2 - 1
V^d_r: -a*dt*x/(2*dx)


In the **lower boundary** there is no $V^d_l$ or $V^d_r$.

In [ ]:
expr_dn, left_gridpoints, right_gridpoints = discretize_crank_nicolson(
    model_expr, V, dV, Vx, Vxx, dx, "dn"
)
coefs_dn = find_coefficients(expr_dn, left_gridpoints + right_gridpoints)

for k, v in coefs_dn.items():
    flip = -1 if k in right_gridpoints else 1
    print(f"{k}: {v * flip}")

V^u_l: -a*dt*x/(2*dx)
V^m_l: a*dt*x/(2*dx) - dt*r/2 - 1
V^d_l: 0
V^u_r: a*dt*x/(2*dx)
V^m_r: -a*dt*x/(2*dx) + dt*r/2 - 1
V^d_r: 0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7b2bac78-17c7-471e-96cd-db7e0e632f5e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>